# SQL Murder Mystery

## Can you find out whodunnit?

![A decorative illustration of a detective looking at an evidence board.](https://mystery.knightlab.com/174092-clue-illustration.png)

There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime.

## SQL sleuths start here

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a **​murder​**that occurred sometime on ​**Jan.15, 2018​** and that it took place in ​**SQL City​**. Start by retrieving the corresponding crime scene report from the police department’s database.

### Exploring the Database Structure

Experienced SQL users can often use database queries to infer the structure of a database. But each database system has different ways of managing this information. The SQL Murder Mystery is built using SQLite. Use this SQL command to find the tables in the Murder Mystery database.

Run this query to find the names of the tables in this database.

This command is specific to SQLite. For other databases, you'll have to learn their specific syntax.


In [1]:
# Install required packages
%pip install ipython-sql sqlalchemy pandas --quiet

# Load SQL magic
%load_ext sql

# Connect to the database
%sql sqlite:///sql-murder-mystery.db

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install prettytable==3.7.0

Note: you may need to restart the kernel to use updated packages.


In [4]:
%%sql
SELECT name
FROM sqlite_master
where type = 'table'

 * sqlite:///sql-murder-mystery.db
Done.


name
crime_scene_report
drivers_license
facebook_event_checkin
interview
get_fit_now_member
get_fit_now_check_in
solution
income
person



Besides knowing the table names, you need to know how each table is structured. The way this works is also dependent upon which database technology you use. Here's how you do it with SQLite.

Run this query to find the structure of the `crime_scene_report` table

Change the value of 'name' to see the structure of the other tables you learned about with the previous query.


In [ ]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'crime_scene_report'

 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE crime_scene_report ( date integer, type text, description text, city text )"


In [ ]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'drivers_license'

 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE drivers_license ( id integer PRIMARY KEY, age integer, height integer, eye_color text, hair_color text, gender text, plate_number text, car_make text, car_model text )"


In [ ]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'facebook_event_checkin'

 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE facebook_event_checkin ( person_id integer, event_id integer, event_name text, date integer, FOREIGN KEY (person_id) REFERENCES person(id) )"


In [ ]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'interview'

 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE interview ( person_id integer, transcript text, FOREIGN KEY (person_id) REFERENCES person(id) )"


In [ ]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'get_fit_now_member'

 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE get_fit_now_member ( id text PRIMARY KEY, person_id integer, name text, membership_start_date integer, membership_status text, FOREIGN KEY (person_id) REFERENCES person(id) )"


In [ ]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'get_fit_now_check_in'

 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE get_fit_now_check_in ( membership_id text, check_in_date integer, check_in_time integer, check_out_time integer, FOREIGN KEY (membership_id) REFERENCES get_fit_now_member(id) )"


In [ ]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'solution'

 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE solution ( user integer, value text )"


In [ ]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'income'

 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE income (ssn CHAR PRIMARY KEY, annual_income integer)"


In [ ]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'person'

 * sqlite:///sql-murder-mystery.db
Done.


sql
"CREATE TABLE person (id integer PRIMARY KEY, name text, license_id integer, address_number integer, address_street_name text, ssn CHAR REFERENCES income (ssn), FOREIGN KEY (license_id) REFERENCES drivers_license (id))"


In [ ]:
%%sql
SELECT *
FROM crime_scene_report
WHERE type = 'murder' AND
date = '20180115' AND
city = 'SQL City'


 * sqlite:///sql-murder-mystery.db
Done.


date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [ ]:
%%sql
SELECT *
FROM person
WHERE name LIKE 'Annabel%' AND address_street_name = 'Franklin Ave'

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [ ]:
%%sql
SELECT *
FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [ ]:
%%sql
SELECT *
FROM interview
WHERE person_id = '14887' -- Morty Schapiro

 * sqlite:///sql-murder-mystery.db
Done.


person_id,transcript
14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


In [ ]:
%%sql
SELECT *
FROM interview
WHERE person_id = '16371' --Annabel Miller

 * sqlite:///sql-murder-mystery.db
Done.


person_id,transcript
16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [ ]:
%%sql
SELECT *
FROM get_fit_now_member
JOIN get_fit_now_check_in
    ON get_fit_now_member.id = get_fit_now_check_in.membership_id
WHERE membership_status = 'gold' AND membership_id LIKE '%48Z%' --AND
    --(person_id = '183779' OR person_id = '423327' OR person_id = '664760')

 * sqlite:///sql-murder-mystery.db
Done.


id,person_id,name,membership_start_date,membership_status,membership_id,check_in_date,check_in_time,check_out_time
48Z7A,28819,Joe Germuska,20160305,gold,48Z7A,20180109,1600,1730
48Z55,67318,Jeremy Bowers,20160101,gold,48Z55,20180109,1530,1700


In [ ]:
%%sql
SELECT *
FROM drivers_license
LEFT JOIN person
ON drivers_license.id = person.license_id
WHERE plate_number LIKE '%H42W%' AND
(person.id = '28819' OR person.id = '67318')

 * sqlite:///sql-murder-mystery.db
Done.


id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id_1,name,license_id,address_number,address_street_name,ssn
423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279



### The rest is up to you!

If you're really comfortable with SQL, you can probably get it from here. To help, here is the schema diagram:

![schema diagram](schema.png)

Use your knowledge of the database schema and SQL commands to find out who committed the murder.
### Check your solution

Did you find the killer? When you think you know the answer, submit your suspect using the following code and find out if you're right.


In [ ]:
%%sql
INSERT INTO solution VALUES (1, 'Jeremy Bowers');
SELECT value FROM solution;

 * sqlite:///sql-murder-mystery.db
1 rows affected.
Done.


value
"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."
